# Create an HTML file to fill a histogram with random numbers

Can generate three types of distributions:

- Distribution of a single six-sided die
- Distribution of a sum of two dice
- Gaussian distribution with a sigma of 1.

Written with a lot of help from ChatGPT



Note, the buttons have some issues on FireFox.  If you open the file locally, you need to load it twice to see them.  If you load it from a website, they just don't show up properly.  You have to mouse over them to see what they do.

02-OCT-2025   E.Prebys   Original

In [ ]:
# These parameters determine the type of distribution and the number of throws  
dist_type = "Gaussian" # Allowed types are "1die", "2dice", and "Gaussian"
throws=1000            # total number of events

# The code near the end determines whether the animation is displayed in the notebook or written to an HTML file
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# Use notebook-friendly animation display
from IPython.display import HTML


# This sets of the configurations for the different types of distributions
nbins={}        # Number of bins in the histogram
hist_range={}   # Range of the histogram
title = {}      # Title of the histogram
ticks = {}      # Where the tick marks are

# Single die
dt='1die'
nbins[dt] = 6
hist_range[dt] = (.5,6.5)
ticks[dt] = np.arange(1,7)
title[dt] = "Random throws of 1 die"

# Two dice
dt='2dice'
nbins[dt] = 11
hist_range[dt] = (1.5,12.5)
ticks[dt] = np.arange(2.,13)
title[dt] = "Random throws of 2 dice"

# Guassian
dt='Gaussian'
nbins[dt]= 20
hist_range[dt]= (-5.,5.)
ticks[dt] = np.arange(-5.,6.)
title[dt] = r'Guassian distribution ($\sigma$=1)'


# For some reason it does two updates before the first frame
# So do this to get the right total
frames = throws - 1 


# Create figure and axis
fig, ax = plt.subplots()
# Need this for 1000 points
plt.rcParams["animation.embed_limit"]=40

# Data storage
all_data = []
ymax = 10.             # maximum bin value (will be updated)

#  This is the routine that will update the histogram 
def update(frame):
    global all_data
    global ymax
    
    # Add new random samples depending on the type
    if dist_type == '1die':                  # Single die
        new_data = [np.random.randint(1,7)]  
    elif dist_type == '2dice':               # 2 dice
        die1 = np.random.randint(1,7)
        die2 = np.random.randint(1,7)
        new_data = [die1+die2]
    else:                                    # Guassian
        new_data = np.random.normal(loc=0, scale=1, size=1)

    # Append the new data onto the data vector
    all_data.extend(new_data)
    
    # Clear and re-draw histogram
    ax.cla()
    counts,bins,patches = ax.hist(all_data, bins=nbins[dist_type], range=hist_range[dist_type], color="skyblue", edgecolor="black")
    
    # Increase the vertical scale if necessary
    if np.array(counts).max() > .9*ymax:
        ymax *= 2
    ax.set_ylim(0, ymax)
    ax.set_title(title[dist_type])
    ax.set_xlabel("Value")
    plt.xticks(ticks[dist_type])

    # Different labeling for Gaussian and dice rolls
    if dist_type=='Gaussian':
        ax.grid()
        # Count the number beyond certain cuts
        arr = np.array(all_data)
        count1 = np.sum((arr<-1.) | (arr>1.))  # \pm 1 sigma
        count2 = np.sum((arr<-2.) | (arr>2.))  # \pm 2 sigma
        count3 = np.sum((arr<-3.) | (arr>3.))  # \pm 3 sigma
        sig = r'$\sigma$'
        display_text = f"Total: {len(all_data)}\n>1 {sig}: {count1}\n>2 {sig}: {count2}\n>3 {sig}: {count3}"
    else:
        display_text = f"Total: {len(all_data)}"
           

    ax.set_ylabel("Count")
    text_obj = ax.text(0.75, 0.95, 
                       display_text,
                       transform=ax.transAxes, ha="left", va="top",
                       fontsize=12, color="darkred",
                       bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.3"))

# Create animation
ani = FuncAnimation(fig, update, frames=frames, interval=20, repeat=False)

# The code fragments below tell whether you want to display it in the notebook or save it as an HTML Page
# Uncomment the one you want.
# Display in notebook
# HTML(ani.to_jshtml())

# These lines save it to a webpage
from matplotlib.animation import HTMLWriter
ani.save(f"{dist_type}.html", writer=HTMLWriter(embed_frames=True, fps=60))